In [ ]:
from datetime import datetime

import polars as pl

import stock

In [ ]:
current_date = datetime.today()

df = stock.kabutan.read_financial_csv(
    stock.PROJECT_ROOT / "data" / "financial" / "9960.csv"
)
df = df.filter(pl.col("annoounce_date") <= current_date)
quarter_df = df.filter(pl.col("duration") == 3).sort("annoounce_date")
year_df = df.filter((pl.col("duration") == 12) & (pl.col("is_prediction") == False)).sort(
    "annoounce_date"
)
pred_df = df.filter(
    (pl.col("is_prediction") is True)
    & (pl.col("duration") == 12)
    & (pl.col("year") >= current_date.year)
).sort("annoounce_date")


In [ ]:
year_df

In [ ]:
def _check_growing(
    df: pl.DataFrame,
    key: str,
    min_growth_rate: float,
    min_duration: int,
    current_date: datetime = datetime.today(),
    num_average: int = 1,
):
    if len(df) < min_duration + num_average:
        return False
    df = df.filter(pl.col("annoounce_date") < current_date).sort(
        "annoounce_date", descending=True, nulls_last=True
    )
    is_growing = True
    for i in range(min_duration):
        cur_data = df[i : i + num_average]
        prev_data = pl.concat(
            [
                df.filter(
                    (pl.col("year") == cur_data["year"][i] - 1)
                    & (pl.col("month") == cur_data["month"][i])
                    & (pl.col("duration") == cur_data["duration"][i])
                )
                for i in range(len(cur_data))
            ]
        )

        print(cur_data)
        print(prev_data)
        if len(cur_data) != len(prev_data):
            print("cur_data != prev_data")
            is_growing = False
            break

        if (not cur_data[key].is_not_null().all()) or (not prev_data[key].is_not_null().all()):
            print("data is null")
            is_growing = False
            break

        cur_val = cur_data[key].sum()
        prev_val = prev_data[key].sum()
        if prev_val > 0:
            is_growing &= cur_val > (1.0 + min_growth_rate) * prev_val
        else:
            is_growing &= cur_val > prev_val * (1.0 - min_growth_rate)
        if not is_growing:
            break
    return is_growing


In [ ]:
_check_growing(
    year_df, "total_revenue", 0.05, 1
)

In [ ]:
year_df